In [1]:
%use lets-plot

In [2]:
val test_input = """7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3
"""

In [21]:
import java.nio.file.Files
import java.nio.file.Paths

data class Vec2(val x: Long, val y: Long)

val use_input = true
val seq = if (use_input) {
    Files.readString(Paths.get("input.txt"))
} else {
    test_input
}.trim().split('\n').asSequence().map { val p = it.split(",")
    Vec2(p[0].toLong(), p[1].toLong())
}.toList()


In [20]:
val maxX = seq.maxOf{it.x}
val maxY = seq.maxOf{it.y}
val minX = seq.minOf{it.x}
val minY = seq.minOf{it.y}
val width = maxX - minX
val height = maxY - minY
println("x: $minX..$maxX")
println("y: $minY..$maxY")
println("Array size would be: $width * $height")

x: 1668..98308
y: 1513..98434
Array size would be: 96640 * 96921


In [19]:
var maxSize = 0L
(0 until seq.size).forEach { i ->
    (i+1 until seq.size).forEach { j ->
        val size = (abs(seq[i].x - seq[j].x) + 1) * (abs(seq[i].y - seq[j].y) + 1)
        if (size > maxSize) maxSize = size
    }
}
println("part 1 answer: ${maxSize.toLong()}")

part 1 answer: 4777409595


In [17]:
import java.awt.Color
import java.awt.Graphics2D
import java.awt.image.BufferedImage

fun createImage(vertices: List<Vec2>): BufferedImage {
    val cellSize = Vec2(5,8)
    val width = vertices.maxOf{ it.x } + 4
    val height = vertices.maxOf{ it.y } + 4
    val image = BufferedImage((width * cellSize.x).toInt(), (height * cellSize.y).toInt(), BufferedImage.TYPE_INT_RGB)
    val g: Graphics2D = image.createGraphics()

    for (y in 0 until height) {
        for (x in 0 until width) {

            g.color = if (vertices.contains(Vec2(x,y))) Color.RED
            else if (isPointInAxisAlignedPolygon(Vec2(x,y), vertices)) Color.GREEN
            else Color.BLACK

            g.fillRect((x * cellSize.x).toInt(), (y * cellSize.y).toInt(), (cellSize.x).toInt(), (cellSize.y).toInt())
        }
    }
    g.dispose()
    return image
}

fun isPointInAxisAlignedPolygon(p: Vec2, vertices: List<Vec2>) : Boolean {
    val y = p.y
    val x = p.x
    val xIntersections = mutableListOf<Long>()

    for (i in vertices.indices) {
        val p1 = vertices[i]
        val p2 = vertices[(i+1) % vertices.size]

        if (p1.y == p2.y && p1.y == y) {
            val minX = minOf(p1.x, p2.x)
            val maxX = maxOf(p1.x, p2.x)
            if (x in minX..maxX) {
                return true
            }
        }

        if (p1.x == p2.x) {
            val minY = minOf(p1.y, p2.y)
            val maxY = maxOf(p1.y, p2.y)
            if (y >= minY && y<maxY) {
                xIntersections.add(p1.x)
            }
        }
    }

    xIntersections.sort()
    for (i in 0 until xIntersections.size step 2) {
        if (i+1 < xIntersections.size) {
            if (x in xIntersections[i]..xIntersections[i+1]) {
                return true
            }
        }
    }
    return false
}

val image = if (width * height < Integer.MAX_VALUE) createImage(seq) else null

image

null

In [16]:
data class RectData(val p1: Vec2, val p2: Vec2, val size: Long)
val allRects = mutableSetOf<RectData>()

(0 until seq.size).forEach { i ->
    (i+1 until seq.size).forEach { j ->
        val size = (abs(seq[i].x - seq[j].x) + 1) * (abs(seq[i].y - seq[j].y) + 1)
        allRects.add(RectData(seq[i], seq[j], size))
    }
}
val allRectsSorted = allRects.sortedByDescending { it.size }
val largest = allRectsSorted.first()
val smallest = allRectsSorted.last()
println("${allRectsSorted.size} rects with sizes between ${smallest.size} to ${largest.size}")



122760 rects with sizes between 5 to 4777409595


In [8]:
class PolygonChecker(vertices: List<Vec2>) {

    private val yToXRanges: Map<Long, List<LongRange>>

    init {
        val minY = vertices.minOf{ it.y }
        val maxY = vertices.maxOf{ it.y }

        val rangesMap = mutableMapOf<Long, List<LongRange>>()

        for(y : Long in minY..maxY) {
            val xIntersections = mutableListOf<Long>()
            val horizontalRanges = mutableListOf<LongRange>()

            for(i in vertices.indices) {
                val p1 = vertices[i]
                val p2 = vertices[(i+1) % vertices.size]

                if (p1.y == p2.y && p1.y == y) {
                    val minX = minOf(p1.x, p2.x)
                    val maxX = maxOf(p1.x, p2.x)
                    horizontalRanges.add(minX..maxX)
                }

                if (p1.x == p2.x) {
                    val minY = minOf(p1.y, p2.y)
                    val maxY = maxOf(p1.y, p2.y)
                    if (y >= minY && y<maxY) {
                        xIntersections.add(p1.x)
                    }
                }
            }

            xIntersections.sort()
            val verticalRanges = mutableListOf<LongRange>()

            for (i in 0 until xIntersections.size step 2) {
                if (i+1 < xIntersections.size) {
                    verticalRanges.add(xIntersections[i]..xIntersections[i+1])
                }
            }
            val allRanges = horizontalRanges + verticalRanges
            if (allRanges.isNotEmpty()) {
                rangesMap[y] = allRanges
            }
        }
        yToXRanges = rangesMap
    }

    fun isPointInside(p: Vec2) : Boolean {
        val ranges = yToXRanges[p.y] ?: return false
        return ranges.any { p.x in it }
    }

    fun isRectInside(p1: Vec2, p2: Vec2) : Boolean {
        val minX = minOf(p1.x, p2.x)
        val maxX = maxOf(p1.x, p2.x)
        val minY = minOf(p1.y, p2.y)
        val maxY = maxOf(p1.y, p2.y)
        return isPointInside(Vec2(minX, minY)) &&
                isPointInside(Vec2(minX, maxY)) &&
                isPointInside(Vec2(maxX, minY)) &&
                isPointInside(Vec2(maxX, maxY)) &&
                (minY..maxY).all { y -> isPointInside(Vec2(minX, y)) && isPointInside(Vec2(minX, y)) } &&
                (minX..maxX).all { x -> isPointInside(Vec2(x, minY)) && isPointInside(Vec2(x, maxY)) }
    }

}

In [14]:
val checker = PolygonChecker(seq)

val bestRect = allRects.sortedByDescending { it.size }.first {
    checker.isRectInside(it.p1, it.p2)
}
bestRect

RectData(p1=Vec2(x=4898, y=66535), p2=Vec2(x=94808, y=50147), size=1473551379)

In [11]:

// Helper function to get rectangle corners closed
fun getRectangleCornersLong(p1: Vec2, p2: Vec2): Pair<Vec2,Vec2> {
    val minX = minOf(p1.x, p2.x)
    val maxX = maxOf(p1.x, p2.x)
    val minY = minOf(p1.y, p2.y)
    val maxY = maxOf(p1.y, p2.y)

    return Vec2(minX, minY) to Vec2(maxX, maxY)
}

// Prepare rectangles with fit info
data class RectInfo(val p1: Vec2, val p2: Vec2, val isGood: Boolean, val size: Long)

// Assuming allRects contains objects with p1 and p2 of type Vec2Long
val rectsInfo = allRects.sortedBy { it.size }.mapNotNull { rect ->
    val fits = checker.isRectInside(rect.p1, rect.p2)
    val s = getRectangleCornersLong(rect.p1, rect.p2)
    if (fits && rect.size > bestRect.size) RectInfo(s.first, s.second, fits, rect.size) else null
}

for (i in rectsInfo.indices) {
    val rect = rectsInfo[i]
    if (rect.p1.x != rect.p2.x && rect.p1.y != rect.p2.y) {
        val plot = letsPlot(mapOf("x" to seq.map { it.x } + seq.first().x, "y" to seq.map { it.y } + seq.first().y)) {
            x = "x"
            y = "y"
        } + geomPolygon(size = 0.3, color = Color.BLUE) { } + geomRect(
            mapOf(
                "xmin" to listOf(rect.p1.x), "ymin" to listOf(rect.p1.y),
                "xmax" to listOf(rect.p2.x), "ymax" to listOf(rect.p2.y)
            ), color = if (rect.isGood) Color.GREEN else Color.RED
        ) {
            xmin = "xmin"
            ymin = "ymin"
            xmax = "xmax"
            ymax = "ymax"
        }  + geomPoint(size = 0.3, color = Color.RED) {}

        ggsave(plot, "frame_${i.toString().padStart(8, '0')}.png")
    }
}